In [ ]:
import pandas as pd
import pulp

#データの取得
s_df = pd.read_csv('students.csv')
s_pair_df = pd.read_csv('student_pairs.csv')

prob = pulp.LpProblem('ClassAssginmentProgram', pulp.LpMaximize)

#生徒のリスト
S = s_df['student_id'].tolist()

#クラスのリスト
C = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

#生徒とクラスのペアのリスト
SC = [(s,c) for s in S for c in C]

#生徒が割り当てれるクラスの定義
x = pulp.LpVariable.dicts('x', SC, cat = 'Binary')

#制約(1) 各生徒は必ず1つのクラスに割り当てられる
for s in S:
    prob += pulp.lpSum(x[s,c] for c in C) == 1
    
#制約(2) 各クラスの生徒数は39人以上、40人以下とする
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S]) >= 39
    prob += pulp.lpSum([x[s,c] for s in S]) <= 40
    
#男子生徒のリスト
S_male = (row.student_id for row in s_df.itertuples() if row.gender == 1)

#女子生徒のリスト
S_female = (row.student_id for row in s_df.itertuples() if row.gender == 0)

#制約(3)各クラスの男子生徒、女子生徒の数をそれぞれ20人以下とする
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_male]) <= 20
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_female]) <= 20

#学力を辞書表現に変換
score = {row.student_id:row.score for row in s_df.itertuples()}

#学年平均を算出
score_mean = s_df['score'].mean()

#制約(4) 各クラスの平均点は学年平均±10点以内
for c in C:
    prob += (score_mean - 10) * pulp.lpSum(x[s,c] for s in S) <= pulp.lpSum(score[s] * x[s,c] for s in S)
    prob += (score_mean + 10) * pulp.lpSum(x[s,c] for s in S) >= pulp.lpSum(score[s] * x[s,c] for s in S)

#リーダー気質の生徒のリスト
S_leader = [row.student_id for row in s_df.itertuples() if row.leader_flag == 1]

#制約(5) 各クラスにリーダー気質の生徒は二人以上
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_leader]) >= 2

#特別支援生徒のリスト
S_support = [row.student_id for row in s_df.itertuples() if row.support_flag == 1]

#制約(6) 各クラスに特別支援生徒は1人以下
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_support]) <= 1

#学力順にクラス編成
class_dic = {1:'A', 2:'B', 3:'C', 4:'D', 5:'E', 6:'F', 7:'G', 0:'H'}
s_df['init_assgined_class'] = s_df['score_rank'].map(lambda x:x % 8).map(class_dic)


#init_flagを作成
init_flag = {(s,c) :0 for s in S for c in C}
for raw in s_df.itertuples():
    init_flag[row.student_id, row.init_assigned_class] = 1

#目的関数:初期クラス編成とできるだけ一致させる
prob += pulp.lpSum([x[s,c] * init_flag[s,c] for s,c in SC])

#求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

#最適化結果の表示
#各クラスに割り当てられている生徒のリストを辞書に格納
C2Ss = {}
for c in C:
    C2Ss[c] = [s for s in S if x[s,c].value()==1]
    
for c, Ss in C2Ss.items():
    print('Class:', c)
    print('Num:', len(Ss))
    print('Student:', Ss)
    print()    